In [1]:
import robotic as ry
import numpy as np
import time

In [15]:
config = ry.Config()
config.addFile("puzzles/p6-wall.g")

config.view()
f = config.getFrame('camera_top')
config.view_setCamera(f)

In [3]:
EGO_NAME = "ego"
OBJ_NAME = "obj"
SUB_GOAL_NAME = "sub-goal1"
GOAL_NAME = "goal"
FLOOR_NAME = "floor"

GAP = config.getFrame(EGO_NAME).getSize()[1] * 2

In [4]:
# manually set the sub goal position
config.getFrame(SUB_GOAL_NAME).setPosition([-1.75, -1.2, 0.2])

In [5]:
def cartesian_to_polar(coordinates):
    x = coordinates[0]
    y = coordinates[1]
    r = np.hypot(x, y)
    theta = np.arctan2(y, x)
    return r, theta

In [6]:
def polar_to_cartesian(coordinates):
    r = coordinates[0]
    theta = coordinates[1]
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return x, y

In [7]:
def solve_touch_with_komo():
    komo = ry.KOMO(config, phases=1, slicesPerPhase=1, kOrder=0, enableCollisions=True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, scale=[1e1])
    komo.addObjective([], ry.FS.negDistance, [EGO_NAME, OBJ_NAME], ry.OT.eq, [1e1])
    ret = ry.NLP_Solver(komo.nlp(), verbose=3).solve()
    if ret:
        path = komo.getPath()
        return path
    else:
        return None

In [8]:
ry.params_clear()
ry.params_add({
    "rrt/stepsize": 0.05,
    "rrt/verbose": 2
})

def solve_rrt(q_begin, q_mid, q_target):
    ITERATIONS = 24
    delta_theta = 2 * np.pi / ITERATIONS
    for i in range(ITERATIONS):
        transpose_polar1 = np.array([GAP, i * delta_theta])
        transpose_cartesian1 = np.array(polar_to_cartesian(transpose_polar1))
        q_home1 = q_mid + transpose_cartesian1
        config.setJointState(q_home1)
        # config.view(pause=True)
        q_sub = solve_touch_with_komo()
        if q_sub is not None:
            config.setJointState(q_begin)
            rrt = ry.PathFinder()
            rrt.setProblem(config, [q_begin], [q_sub])
            solution1 = rrt.solve()
            del rrt
            
            config.setJointState(q_sub)
            config.attach(EGO_NAME, OBJ_NAME)
            q_real_goal = q_target + q_sub - q_mid

            rrt = ry.PathFinder()
            rrt.setProblem(config, [q_sub], [q_real_goal])
            solution2 = rrt.solve()
            config.attach(FLOOR_NAME, OBJ_NAME)
            config.setJointState(q_begin)
            config.view()
            if solution1.feasible and solution2.feasible:
                return q_sub, solution1, solution2

    config.setJointState(q_begin)            
    return None, None, None

In [9]:
q_start = config.getFrame(EGO_NAME).getPosition()[:2]
q_obj = config.getFrame(OBJ_NAME).getPosition()[:2]
q_subgoal = config.getFrame(SUB_GOAL_NAME).getPosition()[:2]
q_goal = config.getFrame(GOAL_NAME).getPosition()[:2]

In [10]:
# move the object from start to the subgoal
q_contact1, solution1, solution2 = solve_rrt(q_start, q_obj, q_subgoal)

for state in solution1.x:
    prev_state = state

    config.setJointState(state)
    config.view()
    config.view_savePng("temp/p6-wall-p1-")
    time.sleep(3./solution1.x.shape[0])

config.attach(EGO_NAME, OBJ_NAME)

for state in solution2.x:
    prev_state = state

    config.setJointState(state)
    config.view()
    config.view_savePng("temp/p6-wall-p2-")
    time.sleep(3./solution2.x.shape[0])

config.attach(FLOOR_NAME, OBJ_NAME)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1 	lambda:[]
----newton---- initial point f(x):0.275759 alpha:1 beta:1
--newton-- it:   1  |Delta|:  0.0522515  alpha:          1  evals:   2  f(y):6.82555e-06  ACCEPT
--newton-- it:   2  |Delta|:0.000259958 	 -- absMax(Delta)<1e-1*o.stopTolerance -- NO UPDATE
==nlp== it:   0  evals:   2  A(x):6.82555e-06  f:          0  g:          0  h: 0.00261257  |x-x'|:  0.0522515 	stop:DeltaConverge 	x:[-0.549739, -1.50968]
==nlp== it:   1  evals:   2  A(x):4.77788e-05  mu:5 	lambda:[0, -0.00522515]
--newton-- it:   3  |Delta|:0.000313196 	 -- absMax(Delta)<1e-1*o.stopTolerance -- NO UPDATE
==nlp== it:   1  evals:   2  A(x):4.77788e-05  f:          0  g:          0  h: 0.00261257  |x-x'|:          0 	stop:DeltaConverge 	x:[-0.549739, -1.50968]
==nlp== StoppingCriterion Delta<0.01
----newton---- final f(x):4.77788e-05
  -- rrt success: queries:7100 tree sizes: 410 153
  path-length=142
-- RRT_PathFinder.cpp:R

In [11]:
q_agent = config.getFrame(EGO_NAME).getPosition()[:2]
q_obj = config.getFrame(OBJ_NAME).getPosition()[:2]

In [12]:
# move the object from subgoal to the goal
q_contact2, solution3, solution4 = solve_rrt(q_agent, q_obj, q_goal)

for state in solution3.x:
    prev_state = state

    config.setJointState(state)
    config.view()
    config.view_savePng("temp/p6-wall-p3-")
    time.sleep(3./solution3.x.shape[0])

config.attach(EGO_NAME, OBJ_NAME)

for state in solution4.x:
    prev_state = state

    config.setJointState(state)
    config.view()
    config.view_savePng("temp/p6-wall-p4-")
    time.sleep(3./solution4.x.shape[0])

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1 	lambda:[]
----newton---- initial point f(x):16.1546 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):     14.532  ACCEPT
--newton-- it:   2  |Delta|:   0.150311  alpha:          1  evals:   3  f(y):    12.2501  ACCEPT
--newton-- it:   3  |Delta|:0.000747814 	 -- absMax(Delta)<1e-1*o.stopTolerance -- NO UPDATE
==nlp== it:   0  evals:   3  A(x):    12.2501  f:          0  g:          0  h:    3.50752  |x-x'|:  0.0496893 	stop:DeltaConverge 	x:[-1.40075, -1.20025]
==nlp== it:   1  evals:   3  A(x):    85.7504  mu:5 	lambda:[7, 0.0150311]
--newton-- it:   4  |Delta|:0.000900963 	 -- absMax(Delta)<1e-1*o.stopTolerance -- NO UPDATE
==nlp== it:   1  evals:   3  A(x):    85.7504  f:          0  g:          0  h:    3.50752  |x-x'|:          0 	stop:DeltaConverge 	x:[-1.40075, -1.20025]
==nlp== StoppingCriterion Delta<0.01
----newton---- final f(x):85.7504
-- R

In [13]:
config.view()

0

In [14]:
config.view_close()
del config